In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold



sns.set()

In [2]:
train = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_KaymcHn.csv')

sample = pd.read_csv('sample_submission_sxfcbdx.csv')

train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [3]:
df = train.append(test, ignore_index=True)

In [4]:
df['gender'] = df['gender'].replace({np.nan: 'Other'}).astype('category')

In [6]:
# df['training_hours2'] = df['training_hours'] ** 2

In [5]:
df['experience'] = df['experience'].replace({'>20':20, '<1':0, np.nan: -9999}).astype(int)

In [6]:
df['last_new_job'] = df['last_new_job'].replace({'>4':5, 'never':0, np.nan: -9999}).astype(int)

In [7]:
df['company_size'] = df['company_size'].replace({'5000-9999': 'multinational',
                                                '1000-4999': 'full-grown',
                                                '10000+':'huge_size',
                                                '50-99':'start-up',
                                                '<10':'start-up',
                                                '10/49':'start-up',
                                                '500-999':'local',
                                                '100-500':'local'}).astype('category')
df['company_size'].value_counts()

start-up         10825
local             6484
huge_size         3666
full-grown        2513
multinational     1062
Name: company_size, dtype: int64

In [9]:
lag_features = ['experience', 'training_hours']

for feat in lag_features:
    for i in range(3):
        df[feat+'_lag'+str(i+1)] = df[feat].shift(i+1)

In [13]:
df.shape

(33380, 24)

In [15]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,...,city_development_index2,city_development_index2_lag1,city_development_index2_lag2,city_development_index2_lag3,experience_lag1,experience_lag2,experience_lag3,training_hours_lag1,training_hours_lag2,training_hours_lag3
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,local,...,0.474721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,start-up,...,0.851929,0.474721,NaN,NaN,3.0,NaN,NaN,106.0,NaN,NaN
2,46,city_16,0.910,Other,Has relevent experience,no_enrollment,Graduate,STEM,6,start-up,...,0.828100,0.851929,0.474721,NaN,14.0,3.0,NaN,69.0,106.0,NaN
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,start-up,...,0.443556,0.828100,0.851929,0.474721,6.0,14.0,3.0,4.0,69.0,106.0
4,21751,city_100,0.887,Other,No relevent experience,no_enrollment,Masters,STEM,8,NaN,...,0.786769,0.443556,0.828100,0.851929,14.0,6.0,14.0,26.0,4.0,69.0


In [219]:
df['target'].value_counts()

0    8296
1     991
Name: target, dtype: int64

<b>Establish a Baseline</b>

In [10]:
# So dummy all cat variables but the city
# city will be label binarized

df['major_discipline'] = df['major_discipline'].replace({np.nan, 'Unknown'})

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
lb = LabelEncoder()
le = LabelBinarizer()
df['city'] = lb.fit_transform(df['city'])
# df['major_discipline'] = le.fit_transform(df['major_discipline'])

In [22]:
df['city'].value_counts()

5      7852
64     3022
48     2928
13     2706
49     1531
       ... 
58        6
108       6
59        6
72        6
32        6
Name: city, Length: 123, dtype: int64

In [11]:
data_dummy = pd.get_dummies(data=df)

data_dummy.shape

(33380, 43)

In [12]:
# normalize training_hours
train_hr_min = data_dummy['training_hours'].min()
train_hr_range = data_dummy['training_hours'].max() - data_dummy['training_hours'].min()
train_hr_scale = (data_dummy['training_hours'] - train_hr_min) / (train_hr_range)

print(train_hr_scale[:2])
data_dummy['training_hours'] = train_hr_scale

# # normalize training_hours2
# train_hr_min2 = data_dummy['training_hours2'].min()
# train_hr_range2 = data_dummy['training_hours2'].max() - data_dummy['training_hours2'].min()
# train_hr_scale2 = (data_dummy['training_hours2'] - train_hr_min2) / (train_hr_range2)

# print(train_hr_scale[:2])
# data_dummy['training_hours2'] = train_hr_scale2

# normalize experience 
exp_min = data_dummy['experience'].min()
exp_range = data_dummy['experience'].max() - data_dummy['experience'].min()
exp_scale = (data_dummy['experience'] - exp_min) / exp_range

data_dummy['experience'] = exp_scale

# normalize experience2 
# exp_min2 = data_dummy['experience2'].min()
# exp_range2 = data_dummy['experience2'].max() - data_dummy['experience2'].min()
# exp_scale2 = (data_dummy['experience2'] - exp_min2) / exp_range2

# data_dummy['experience2'] = exp_scale2


0    0.313433
1    0.202985
Name: training_hours, dtype: float64


In [13]:
data_dummy['target'].isna().sum()

15021

In [22]:
corr = data_dummy.corr()['target']
corr.tail(15)

major_discipline_Humanities         0.001799
major_discipline_No Major          -0.011012
major_discipline_Other             -0.009869
major_discipline_STEM               0.023122
company_size_full-grown            -0.019276
company_size_huge_size             -0.011757
company_size_local                 -0.036676
company_size_multinational         -0.003704
company_size_start-up              -0.044483
company_type_Early Stage Startup   -0.002640
company_type_Funded Startup        -0.028633
company_type_NGO                   -0.001469
company_type_Other                  0.006578
company_type_Public Sector          0.014517
company_type_Pvt Ltd               -0.074536
Name: target, dtype: float64

In [116]:
# from sklearn.feature_selection import SelectKBest, chi2

In [27]:
corr.sort_values()

sin_city_development_index                    -0.135700
city_development_index                        -0.135700
city_development_index2                       -0.133118
enrolled_university_no_enrollment             -0.079690
company_type_Pvt Ltd                          -0.074536
relevent_experience_Has relevent experience   -0.074088
company_size_start-up                         -0.044483
gender_Male                                   -0.043541
company_size_local                            -0.036676
company_type_Funded Startup                   -0.028633
last_new_job                                  -0.024771
education_level_Masters                       -0.021804
education_level_Primary School                -0.021615
company_size_full-grown                       -0.019276
education_level_High School                   -0.014051
experience                                    -0.012974
company_size_huge_size                        -0.011757
major_discipline_No Major                     -0

In [14]:
# separate into train set and test set
train_df = data_dummy[data_dummy['target'].isna() == False]
test_df = data_dummy[data_dummy['target'].isna() == True]

print(train_df.shape)
print(test_df.shape)

(18359, 43)
(15021, 43)


In [15]:
test_df.drop('target', axis=1, inplace=True)
print('Dropped Target column from test data')

Dropped Target column from test data


In [16]:
train_df.head()

,enrollee_id,city,city_development_index,experience,last_new_job,training_hours,target,experience_lag1,experience_lag2,experience_lag3,...,company_size_huge_size,company_size_local,company_size_multinational,company_size_start-up,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd
0,23798,41,0.689,0.998303,1,0.313433,0.0,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
1,29166,112,0.923,0.999401,1,0.202985,0.0,3.0,NaN,NaN,...,0,0,0,1,0,1,0,0,0,0
2,46,48,0.910,0.998603,2,0.008955,0.0,14.0,3.0,NaN,...,0,0,0,1,0,0,0,0,1,0
3,18527,93,0.666,0.999401,1,0.074627,0.0,6.0,14.0,3.0,...,0,0,0,1,0,0,0,0,0,1
4,21751,2,0.887,0.998802,2,0.259701,1.0,14.0,6.0,14.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
ID = test_df['enrollee_id']
test_df.drop('enrollee_id', axis=1, inplace=True)
train_df.drop('enrollee_id', axis=1, inplace=True)


In [18]:
# Split train data for eval
X = train_df.drop('target', axis=1)
Y = train_df['target'].astype(int)

In [34]:
# DownSample Train to fix data Imbalance
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X_rus, Y_rus = rus.fit_sample(X, Y)

Using TensorFlow backend.


In [107]:
# DownSample Train to fix data Imbalance
from imblearn.over_sampling import RandomOverSampler
ada = RandomOverSampler()
X_rus, Y_rus = ada.fit_sample(X, Y)

In [25]:
train_df['target'].value_counts()

0.0    15934
1.0     2425
Name: target, dtype: int64

In [122]:
# ks = SelectKBest(chi2, k=30)
# X_new = ks.fit_transform(X, Y)

In [123]:
# mask = ks.get_support()
# selected_features = []
# for boo, feature in zip(mask, data_dummy.columns):
#     if boo:
#         selected_features.append(feature)
# print(selected_features)

['city', 'training_hours', 'target', 'gender_Male', 'gender_Other', 'relevent_experience_Has relevent experience', 'relevent_experience_No relevent experience', 'enrolled_university_Full time course', 'enrolled_university_Part time course', 'enrolled_university_no_enrollment', 'education_level_Graduate', 'education_level_Masters', 'major_discipline_Other', 'experience_10', 'experience_14', 'experience_15', 'experience_16', 'experience_18', 'experience_2', 'experience_20', 'experience_8', 'experience_9', 'experience_>20', 'company_size_10/49', 'company_size_1000-4999', 'company_size_500-999', 'company_size_<10', 'company_type_NGO', 'company_type_Other', 'last_new_job_3']


In [19]:
X = X.fillna(-1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=2020)

In [21]:
train_auc_values, test_auc_values = {}, {}

In [29]:
# lr = LogisticRegression()
# lr.fit(X_train, y_train)
# y_pred_lr = lr.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [34]:
# how did our model perform on the test set?
count_misclassified = (y_test != y_pred_lr).sum()
print('Misclassified samples: {}'.format(count_misclassified))
acc = accuracy_score(y_pred_lr, y_test)
print('Accuracy: {:.2f}'.format(acc))

Misclassified samples: 491
Accuracy: 0.87


In [65]:
from sklearn.model_selection import cross_val_score
# scores = cross_val_score(lr, X, Y, cv=7, scoring='roc_auc').mean()
# print('CV Score:', (scores))

In [67]:
# Gradient Boosting

gbr_params = {
    'n_estimators': 200,
#     'criterion': 'auc',
    'max_features': 'sqrt',
    'max_depth': 3,
    'min_samples_leaf': 10,
    'random_state': 2020
}

grb = GradientBoostingClassifier(**gbr_params)
grb.fit(X_train, y_train)
grb_preds_train = grb.predict(X_train)
grb_preds_test = grb.predict(X_test)

train_auc_values['grb'] = roc_auc_score(y_train, grb_preds_train)
test_auc_values['grb'] = roc_auc_score(y_test, grb_preds_test)

In [68]:
scores = cross_val_score(grb, X, Y, cv=7, scoring='roc_auc').mean()
print('CV Score:', (scores))

CV Score: 0.6495639260565337


In [26]:
# XGBClassifier

xgb_params = {
    'base_score': 0.7,
    'n_estimators': 200,
    'verbose': 300,
    'max_depth': 5,
    'learning_rate': 0.03,
    'objective':'binary:logistic',
    'subsample': 0.8
}
xgb = XGBClassifier(**xgb_params)
eval_set  = [(X_train,y_train), (X_test,y_test)]
xgb.fit(X_train, y_train, eval_set=eval_set, eval_metric='auc', early_stopping_rounds=50, verbose=50)
xgb_preds_train = xgb.predict(X_train)
xgb_preds_test = xgb.predict(X_test)

train_auc_values['xgb'] = roc_auc_score(y_train, xgb_preds_train)
test_auc_values['xgb'] = roc_auc_score(y_test, xgb_preds_test)

[0]	validation_0-auc:0.635786	validation_1-auc:0.613055
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[50]	validation_0-auc:0.71938	validation_1-auc:0.63466
Stopping. Best iteration:
[46]	validation_0-auc:0.716771	validation_1-auc:0.635887



In [49]:
# testing new models
# clf = XGBClassifier(base_score=0.7,n_estimators=200, verbose=300,
#                     max_depth=5,learning_rate=0.03,objective='binary:logistic',subsample=0.8)
# eval_set  = [(X_train,y_train), (X_test,y_test)]
# #.900057
# clf.fit(X_train, y_train, eval_set=eval_set,eval_metric="auc",
#         early_stopping_rounds=50, verbose=50)

[0]	validation_0-auc:0.639126	validation_1-auc:0.609578
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[50]	validation_0-auc:0.723148	validation_1-auc:0.634892
[100]	validation_0-auc:0.756639	validation_1-auc:0.635443
Stopping. Best iteration:
[62]	validation_0-auc:0.732668	validation_1-auc:0.636573



XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.03, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbose=300, verbosity=1)

In [69]:
scores = cross_val_score(xgb, X, Y, cv=7, scoring='roc_auc').mean()
print('CV Score:', (scores))

CV Score: 0.6528639392761706


In [28]:
cats_params = {
    'learning_rate': 0.01,
    'eval_metric': 'AUC',
    'depth': 7,
    'iterations': 2000,
    'verbose': 300,
    'early_stopping_rounds': 100
}
eval_dataset = Pool(X_test, y_test)

cats = CatBoostClassifier(**cats_params)
cats.fit(X_train, y_train, eval_set=eval_dataset)
cats_preds_train = cats.predict(X_train)
cats_preds_test = cats.predict(X_test)

train_auc_values['cats'] = roc_auc_score(y_train, cats_preds_train)
test_auc_values['cats'] = roc_auc_score(y_test, cats_preds_test)

0:	test: 0.5871064	best: 0.5871064 (0)	total: 121ms	remaining: 4m 2s
300:	test: 0.6370033	best: 0.6374067 (272)	total: 8.66s	remaining: 48.9s
600:	test: 0.6394709	best: 0.6395317 (599)	total: 16.8s	remaining: 39.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6395316899
bestIteration = 599

Shrink model to first 600 iterations.


In [34]:

# eval_dataset = Pool(X_test, y_test)

# cats_model = CatBoostClassifier(learning_rate=0.01,eval_metric='AUC',depth=7, iterations=2000)

# cats_model.fit(X_train, y_train, eval_set=eval_dataset, verbose=300,early_stopping_rounds=100)

0:	test: 0.5310067	best: 0.5310067 (0)	total: 133ms	remaining: 4m 26s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6343116685
bestIteration = 18

Shrink model to first 19 iterations.


In [35]:
scores = cross_val_score(cats_model, X, Y, cv=7, scoring='roc_auc', verbose=300).mean()
print('CV Score:', (scores))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  ................................................................
0:	total: 12.9ms	remaining: 25.9s
1:	total: 30.3ms	remaining: 30.3s
2:	total: 44.6ms	remaining: 29.7s
3:	total: 59.8ms	remaining: 29.9s
4:	total: 72.7ms	remaining: 29s
5:	total: 86.4ms	remaining: 28.7s
6:	total: 103ms	remaining: 29.3s
7:	total: 116ms	remaining: 29s
8:	total: 132ms	remaining: 29.3s
9:	total: 147ms	remaining: 29.2s
10:	total: 161ms	remaining: 29.1s
11:	total: 183ms	remaining: 30.4s
12:	total: 197ms	remaining: 30.1s
13:	total: 212ms	remaining: 30s
14:	total: 225ms	remaining: 29.8s
15:	total: 244ms	remaining: 30.2s
16:	total: 259ms	remaining: 30.3s
17:	total: 274ms	remaining: 30.1s
18:	total: 288ms	remaining: 30s
19:	total: 302ms	remaining: 29.9s
20:	total: 329ms	remaining: 31s
21:	total: 344ms	remaining: 31s
22:	total: 360ms	remaining: 30.9s
23:	total: 375ms	remaining: 30.9s
24:	total: 389ms	remaining: 30.7s
25:	total: 406

242:	total: 3.95s	remaining: 28.5s
243:	total: 3.96s	remaining: 28.5s
244:	total: 3.98s	remaining: 28.5s
245:	total: 4s	remaining: 28.5s
246:	total: 4.01s	remaining: 28.5s
247:	total: 4.03s	remaining: 28.5s
248:	total: 4.04s	remaining: 28.4s
249:	total: 4.06s	remaining: 28.4s
250:	total: 4.08s	remaining: 28.4s
251:	total: 4.09s	remaining: 28.4s
252:	total: 4.11s	remaining: 28.4s
253:	total: 4.13s	remaining: 28.4s
254:	total: 4.14s	remaining: 28.4s
255:	total: 4.16s	remaining: 28.3s
256:	total: 4.17s	remaining: 28.3s
257:	total: 4.19s	remaining: 28.3s
258:	total: 4.21s	remaining: 28.3s
259:	total: 4.22s	remaining: 28.2s
260:	total: 4.24s	remaining: 28.2s
261:	total: 4.25s	remaining: 28.2s
262:	total: 4.26s	remaining: 28.2s
263:	total: 4.28s	remaining: 28.1s
264:	total: 4.29s	remaining: 28.1s
265:	total: 4.31s	remaining: 28.1s
266:	total: 4.32s	remaining: 28s
267:	total: 4.33s	remaining: 28s
268:	total: 4.35s	remaining: 28s
269:	total: 4.37s	remaining: 28s
270:	total: 4.38s	remaining: 27

482:	total: 7.81s	remaining: 24.5s
483:	total: 7.83s	remaining: 24.5s
484:	total: 7.84s	remaining: 24.5s
485:	total: 7.85s	remaining: 24.5s
486:	total: 7.87s	remaining: 24.4s
487:	total: 7.88s	remaining: 24.4s
488:	total: 7.9s	remaining: 24.4s
489:	total: 7.91s	remaining: 24.4s
490:	total: 7.93s	remaining: 24.4s
491:	total: 7.94s	remaining: 24.3s
492:	total: 7.95s	remaining: 24.3s
493:	total: 7.97s	remaining: 24.3s
494:	total: 7.99s	remaining: 24.3s
495:	total: 8s	remaining: 24.3s
496:	total: 8.01s	remaining: 24.2s
497:	total: 8.03s	remaining: 24.2s
498:	total: 8.05s	remaining: 24.2s
499:	total: 8.06s	remaining: 24.2s
500:	total: 8.09s	remaining: 24.2s
501:	total: 8.1s	remaining: 24.2s
502:	total: 8.12s	remaining: 24.2s
503:	total: 8.13s	remaining: 24.1s
504:	total: 8.14s	remaining: 24.1s
505:	total: 8.16s	remaining: 24.1s
506:	total: 8.17s	remaining: 24.1s
507:	total: 8.19s	remaining: 24s
508:	total: 8.2s	remaining: 24s
509:	total: 8.21s	remaining: 24s
510:	total: 8.23s	remaining: 24s

723:	total: 11.6s	remaining: 20.5s
724:	total: 11.6s	remaining: 20.4s
725:	total: 11.6s	remaining: 20.4s
726:	total: 11.7s	remaining: 20.4s
727:	total: 11.7s	remaining: 20.4s
728:	total: 11.7s	remaining: 20.4s
729:	total: 11.7s	remaining: 20.4s
730:	total: 11.7s	remaining: 20.3s
731:	total: 11.7s	remaining: 20.3s
732:	total: 11.7s	remaining: 20.3s
733:	total: 11.8s	remaining: 20.3s
734:	total: 11.8s	remaining: 20.3s
735:	total: 11.8s	remaining: 20.3s
736:	total: 11.8s	remaining: 20.2s
737:	total: 11.8s	remaining: 20.2s
738:	total: 11.8s	remaining: 20.2s
739:	total: 11.9s	remaining: 20.2s
740:	total: 11.9s	remaining: 20.2s
741:	total: 11.9s	remaining: 20.2s
742:	total: 11.9s	remaining: 20.1s
743:	total: 11.9s	remaining: 20.1s
744:	total: 11.9s	remaining: 20.1s
745:	total: 12s	remaining: 20.1s
746:	total: 12s	remaining: 20.1s
747:	total: 12s	remaining: 20.1s
748:	total: 12s	remaining: 20s
749:	total: 12s	remaining: 20s
750:	total: 12s	remaining: 20s
751:	total: 12s	remaining: 20s
752:	to

970:	total: 15.5s	remaining: 16.4s
971:	total: 15.5s	remaining: 16.4s
972:	total: 15.5s	remaining: 16.3s
973:	total: 15.5s	remaining: 16.3s
974:	total: 15.5s	remaining: 16.3s
975:	total: 15.5s	remaining: 16.3s
976:	total: 15.5s	remaining: 16.3s
977:	total: 15.6s	remaining: 16.3s
978:	total: 15.6s	remaining: 16.3s
979:	total: 15.6s	remaining: 16.2s
980:	total: 15.6s	remaining: 16.2s
981:	total: 15.6s	remaining: 16.2s
982:	total: 15.6s	remaining: 16.2s
983:	total: 15.7s	remaining: 16.2s
984:	total: 15.7s	remaining: 16.1s
985:	total: 15.7s	remaining: 16.1s
986:	total: 15.7s	remaining: 16.1s
987:	total: 15.7s	remaining: 16.1s
988:	total: 15.7s	remaining: 16.1s
989:	total: 15.8s	remaining: 16.1s
990:	total: 15.8s	remaining: 16.1s
991:	total: 15.8s	remaining: 16s
992:	total: 15.8s	remaining: 16s
993:	total: 15.8s	remaining: 16s
994:	total: 15.8s	remaining: 16s
995:	total: 15.8s	remaining: 16s
996:	total: 15.9s	remaining: 16s
997:	total: 15.9s	remaining: 15.9s
998:	total: 15.9s	remaining: 15.

...

908:	total: 14.8s	remaining: 17.8s
909:	total: 14.8s	remaining: 17.8s
910:	total: 14.8s	remaining: 17.7s
911:	total: 14.8s	remaining: 17.7s
912:	total: 14.9s	remaining: 17.7s
913:	total: 14.9s	remaining: 17.7s
914:	total: 14.9s	remaining: 17.7s
915:	total: 14.9s	remaining: 17.6s
916:	total: 14.9s	remaining: 17.6s
917:	total: 14.9s	remaining: 17.6s
918:	total: 15s	remaining: 17.6s
919:	total: 15s	remaining: 17.6s
920:	total: 15s	remaining: 17.6s
921:	total: 15s	remaining: 17.5s
922:	total: 15s	remaining: 17.5s
923:	total: 15s	remaining: 17.5s
924:	total: 15s	remaining: 17.5s
925:	total: 15.1s	remaining: 17.5s
926:	total: 15.1s	remaining: 17.4s
927:	total: 15.1s	remaining: 17.4s
928:	total: 15.1s	remaining: 17.4s
929:	total: 15.1s	remaining: 17.4s
930:	total: 15.1s	remaining: 17.4s
931:	total: 15.1s	remaining: 17.3s
932:	total: 15.2s	remaining: 17.3s
933:	total: 15.2s	remaining: 17.3s
934:	total: 15.2s	remaining: 17.3s
935:	total: 15.2s	remaining: 17.3s
936:	total: 15.2s	remaining: 17.3s

1143:	total: 18.5s	remaining: 13.8s
1144:	total: 18.5s	remaining: 13.8s
1145:	total: 18.5s	remaining: 13.8s
1146:	total: 18.5s	remaining: 13.8s
1147:	total: 18.5s	remaining: 13.8s
1148:	total: 18.6s	remaining: 13.7s
1149:	total: 18.6s	remaining: 13.7s
1150:	total: 18.6s	remaining: 13.7s
1151:	total: 18.6s	remaining: 13.7s
1152:	total: 18.6s	remaining: 13.7s
1153:	total: 18.6s	remaining: 13.7s
1154:	total: 18.6s	remaining: 13.6s
1155:	total: 18.7s	remaining: 13.6s
1156:	total: 18.7s	remaining: 13.6s
1157:	total: 18.7s	remaining: 13.6s
1158:	total: 18.7s	remaining: 13.6s
1159:	total: 18.7s	remaining: 13.6s
1160:	total: 18.7s	remaining: 13.5s
1161:	total: 18.8s	remaining: 13.5s
1162:	total: 18.8s	remaining: 13.5s
1163:	total: 18.8s	remaining: 13.5s
1164:	total: 18.8s	remaining: 13.5s
1165:	total: 18.8s	remaining: 13.5s
1166:	total: 18.8s	remaining: 13.4s
1167:	total: 18.9s	remaining: 13.4s
1168:	total: 18.9s	remaining: 13.4s
1169:	total: 18.9s	remaining: 13.4s
1170:	total: 18.9s	remaining

1373:	total: 22.1s	remaining: 10.1s
1374:	total: 22.1s	remaining: 10.1s
1375:	total: 22.2s	remaining: 10s
1376:	total: 22.2s	remaining: 10s
1377:	total: 22.2s	remaining: 10s
1378:	total: 22.2s	remaining: 9.99s
1379:	total: 22.2s	remaining: 9.98s
1380:	total: 22.2s	remaining: 9.96s
1381:	total: 22.2s	remaining: 9.94s
1382:	total: 22.3s	remaining: 9.93s
1383:	total: 22.3s	remaining: 9.91s
1384:	total: 22.3s	remaining: 9.9s
1385:	total: 22.3s	remaining: 9.88s
1386:	total: 22.3s	remaining: 9.86s
1387:	total: 22.3s	remaining: 9.85s
1388:	total: 22.3s	remaining: 9.83s
1389:	total: 22.4s	remaining: 9.81s
1390:	total: 22.4s	remaining: 9.8s
1391:	total: 22.4s	remaining: 9.78s
1392:	total: 22.4s	remaining: 9.76s
1393:	total: 22.4s	remaining: 9.75s
1394:	total: 22.4s	remaining: 9.73s
1395:	total: 22.5s	remaining: 9.71s
1396:	total: 22.5s	remaining: 9.7s
1397:	total: 22.5s	remaining: 9.69s
1398:	total: 22.5s	remaining: 9.67s
1399:	total: 22.5s	remaining: 9.65s
1400:	total: 22.5s	remaining: 9.64s
1

1614:	total: 25.9s	remaining: 6.18s
1615:	total: 26s	remaining: 6.17s
1616:	total: 26s	remaining: 6.15s
1617:	total: 26s	remaining: 6.13s
1618:	total: 26s	remaining: 6.12s
1619:	total: 26s	remaining: 6.1s
1620:	total: 26s	remaining: 6.09s
1621:	total: 26.1s	remaining: 6.07s
1622:	total: 26.1s	remaining: 6.06s
1623:	total: 26.1s	remaining: 6.04s
1624:	total: 26.1s	remaining: 6.02s
1625:	total: 26.1s	remaining: 6.01s
1626:	total: 26.1s	remaining: 5.99s
1627:	total: 26.2s	remaining: 5.98s
1628:	total: 26.2s	remaining: 5.96s
1629:	total: 26.2s	remaining: 5.94s
1630:	total: 26.2s	remaining: 5.93s
1631:	total: 26.2s	remaining: 5.91s
1632:	total: 26.2s	remaining: 5.9s
1633:	total: 26.3s	remaining: 5.88s
1634:	total: 26.3s	remaining: 5.86s
1635:	total: 26.3s	remaining: 5.85s
1636:	total: 26.3s	remaining: 5.83s
1637:	total: 26.3s	remaining: 5.82s
1638:	total: 26.3s	remaining: 5.8s
1639:	total: 26.4s	remaining: 5.78s
1640:	total: 26.4s	remaining: 5.77s
1641:	total: 26.4s	remaining: 5.75s
1642:	t

1846:	total: 29.6s	remaining: 2.45s
1847:	total: 29.6s	remaining: 2.44s
1848:	total: 29.6s	remaining: 2.42s
1849:	total: 29.7s	remaining: 2.4s
1850:	total: 29.7s	remaining: 2.39s
1851:	total: 29.7s	remaining: 2.37s
1852:	total: 29.7s	remaining: 2.36s
1853:	total: 29.7s	remaining: 2.34s
1854:	total: 29.7s	remaining: 2.33s
1855:	total: 29.8s	remaining: 2.31s
1856:	total: 29.8s	remaining: 2.29s
1857:	total: 29.8s	remaining: 2.28s
1858:	total: 29.8s	remaining: 2.26s
1859:	total: 29.8s	remaining: 2.24s
1860:	total: 29.8s	remaining: 2.23s
1861:	total: 29.9s	remaining: 2.21s
1862:	total: 29.9s	remaining: 2.2s
1863:	total: 29.9s	remaining: 2.18s
1864:	total: 29.9s	remaining: 2.16s
1865:	total: 29.9s	remaining: 2.15s
1866:	total: 29.9s	remaining: 2.13s
1867:	total: 30s	remaining: 2.12s
1868:	total: 30s	remaining: 2.1s
1869:	total: 30s	remaining: 2.08s
1870:	total: 30s	remaining: 2.07s
1871:	total: 30s	remaining: 2.05s
1872:	total: 30s	remaining: 2.04s
1873:	total: 30s	remaining: 2.02s
1874:	tot

In [30]:
lgbm_params = {
    'learning_rate': 0.02,
    'eval_metric': 'AUC',
    'max_depth': 5,
    'n_estimators': 600,
    'verbose': 100,
    'early_stopping_rounds': 100,
    'boosting_type': 'dart'
}

lgbm = LGBMClassifier(**lgbm_params)
lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)])
lgbm_preds_train = lgbm.predict(X_train)
lgbm_preds_test = lgbm.predict(X_test)

train_auc_values['lgbm'] = roc_auc_score(y_train, lgbm_preds_train)
test_auc_values['lgbm'] = roc_auc_score(y_test, lgbm_preds_test)

[1]	valid_0's binary_logloss: 0.39269
[2]	valid_0's binary_logloss: 0.392013
[3]	valid_0's binary_logloss: 0.391355
[4]	valid_0's binary_logloss: 0.390772
[5]	valid_0's binary_logloss: 0.390181
[6]	valid_0's binary_logloss: 0.389648
[7]	valid_0's binary_logloss: 0.389164
[8]	valid_0's binary_logloss: 0.389357
[9]	valid_0's binary_logloss: 0.388844
[10]	valid_0's binary_logloss: 0.388417
[11]	valid_0's binary_logloss: 0.387976
[12]	valid_0's binary_logloss: 0.388138
[13]	valid_0's binary_logloss: 0.387748
[14]	valid_0's binary_logloss: 0.387327
[15]	valid_0's binary_logloss: 0.386946
[16]	valid_0's binary_logloss: 0.386574
[17]	valid_0's binary_logloss: 0.386235
[18]	valid_0's binary_logloss: 0.385896
[19]	valid_0's binary_logloss: 0.385615
[20]	valid_0's binary_logloss: 0.38528
[21]	valid_0's binary_logloss: 0.385377
[22]	valid_0's binary_logloss: 0.385108
[23]	valid_0's binary_logloss: 0.384799
[24]	valid_0's binary_logloss: 0.384459
[25]	valid_0's binary_logloss: 0.38418
[26]	valid_0

[205]	valid_0's binary_logloss: 0.410024
[206]	valid_0's binary_logloss: 0.408953
[207]	valid_0's binary_logloss: 0.409402
[208]	valid_0's binary_logloss: 0.408358
[209]	valid_0's binary_logloss: 0.408887
[210]	valid_0's binary_logloss: 0.409262
[211]	valid_0's binary_logloss: 0.409749
[212]	valid_0's binary_logloss: 0.408661
[213]	valid_0's binary_logloss: 0.407627
[214]	valid_0's binary_logloss: 0.406675
[215]	valid_0's binary_logloss: 0.405726
[216]	valid_0's binary_logloss: 0.406161
[217]	valid_0's binary_logloss: 0.405209
[218]	valid_0's binary_logloss: 0.404312
[219]	valid_0's binary_logloss: 0.406745
[220]	valid_0's binary_logloss: 0.405773
[221]	valid_0's binary_logloss: 0.40629
[222]	valid_0's binary_logloss: 0.405365
[223]	valid_0's binary_logloss: 0.404454
[224]	valid_0's binary_logloss: 0.404917
[225]	valid_0's binary_logloss: 0.405495
[226]	valid_0's binary_logloss: 0.405924
[227]	valid_0's binary_logloss: 0.406253
[228]	valid_0's binary_logloss: 0.405293
[229]	valid_0's b

[406]	valid_0's binary_logloss: 0.38492
[407]	valid_0's binary_logloss: 0.384709
[408]	valid_0's binary_logloss: 0.38488
[409]	valid_0's binary_logloss: 0.384731
[410]	valid_0's binary_logloss: 0.384577
[411]	valid_0's binary_logloss: 0.384347
[412]	valid_0's binary_logloss: 0.384504
[413]	valid_0's binary_logloss: 0.38467
[414]	valid_0's binary_logloss: 0.384469
[415]	valid_0's binary_logloss: 0.38431
[416]	valid_0's binary_logloss: 0.38409
[417]	valid_0's binary_logloss: 0.384252
[418]	valid_0's binary_logloss: 0.384417
[419]	valid_0's binary_logloss: 0.384221
[420]	valid_0's binary_logloss: 0.384373
[421]	valid_0's binary_logloss: 0.384156
[422]	valid_0's binary_logloss: 0.384021
[423]	valid_0's binary_logloss: 0.383849
[424]	valid_0's binary_logloss: 0.383696
[425]	valid_0's binary_logloss: 0.383852
[426]	valid_0's binary_logloss: 0.383682
[427]	valid_0's binary_logloss: 0.383489
[428]	valid_0's binary_logloss: 0.383632
[429]	valid_0's binary_logloss: 0.383453
[430]	valid_0's binar

In [36]:
# lgbm = LGBMClassifier(boosting_type="dart", learning_rate=0.02, max_depth=5,n_estimators=700)

# lgbm.fit(X_train, y_train,eval_metric='auc',eval_set=[(X_test, y_test)], verbose=100,
#          early_stopping_rounds=100)

[100]	valid_0's auc: 0.624758	valid_0's binary_logloss: 0.407145
[200]	valid_0's auc: 0.628733	valid_0's binary_logloss: 0.405904
[300]	valid_0's auc: 0.628949	valid_0's binary_logloss: 0.390971
[400]	valid_0's auc: 0.628766	valid_0's binary_logloss: 0.384226
[500]	valid_0's auc: 0.628698	valid_0's binary_logloss: 0.381681
[600]	valid_0's auc: 0.628898	valid_0's binary_logloss: 0.381755
[700]	valid_0's auc: 0.628613	valid_0's binary_logloss: 0.381086


LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.02, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=700, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [37]:
scores = cross_val_score(lgbm, X, Y, cv=7, scoring='roc_auc').mean()
print('CV Score:', (scores))

CV Score: 0.6497307611884694


## Stacked Models

In [31]:
num_base_predictors = len(train_auc_values) # 4

x_train_with_metapreds = np.zeros((X_train.shape[0], X_train.shape[1]+num_base_predictors))
x_train_with_metapreds[:, :-num_base_predictors] = X_train
x_train_with_metapreds[:, -num_base_predictors:] = -1

In [34]:
kf = KFold(n_splits=5, random_state=11)

for train_indices, val_indices in kf.split(X_train):
    kfold_x_train, kfold_x_val = X_train.iloc[train_indices], X_train.iloc[val_indices]
    kfold_y_train, kfold_y_val = y_train.iloc[train_indices], y_train.iloc[val_indices]
    
    predictions = []
    
    dt = GradientBoostingClassifier(**gbr_params)
    dt.fit(kfold_x_train, kfold_y_train)
    predictions.append(dt.predict(kfold_x_val))

    knn = XGBClassifier(**xgb_params)
    knn.fit(kfold_x_train, kfold_y_train)
    predictions.append(knn.predict(kfold_x_val))

    gbr = CatBoostClassifier(**cats_params)
    gbr.fit(kfold_x_train, kfold_y_train)
    predictions.append(gbr.predict(kfold_x_val))
    
    lgbm = LGBMClassifier(**lgbm_params)
    lgbm.fit(kfold_x_train, kfold_y_train)
    predictions.append(lgbm.predict(kfold_x_val))
    
    for i, preds in enumerate(predictions):
        x_train_with_metapreds[val_indices, -(i+1)] = preds

0:	total: 50.8ms	remaining: 1m 41s
300:	total: 7.13s	remaining: 40.2s
600:	total: 13.9s	remaining: 32.4s
900:	total: 20.7s	remaining: 25.3s
1200:	total: 27.6s	remaining: 18.3s
1500:	total: 34.3s	remaining: 11.4s
1800:	total: 41.1s	remaining: 4.54s
1999:	total: 45.6s	remaining: 0us
0:	total: 48.7ms	remaining: 1m 37s
300:	total: 7.26s	remaining: 41s
600:	total: 14.6s	remaining: 33.9s
900:	total: 21.2s	remaining: 25.9s
1200:	total: 27.9s	remaining: 18.6s
1500:	total: 34.6s	remaining: 11.5s
1800:	total: 41.2s	remaining: 4.55s
1999:	total: 45.8s	remaining: 0us
0:	total: 27.7ms	remaining: 55.3s
300:	total: 7.07s	remaining: 39.9s
600:	total: 14.2s	remaining: 33s
900:	total: 21.1s	remaining: 25.8s
1200:	total: 27.9s	remaining: 18.5s
1500:	total: 34.6s	remaining: 11.5s
1800:	total: 41.5s	remaining: 4.58s
1999:	total: 46.2s	remaining: 0us
0:	total: 21.9ms	remaining: 43.9s
300:	total: 7.2s	remaining: 40.7s
600:	total: 14.5s	remaining: 33.7s
900:	total: 21.8s	remaining: 26.6s
1200:	total: 29.4s	re

In [42]:
x_val_with_metapreds = np.zeros((X_test.shape[0], X_test.shape[1]+num_base_predictors))
x_val_with_metapreds[:, :-num_base_predictors] = X_test
x_val_with_metapreds[:, -num_base_predictors:] = -1

In [45]:
predictions = []
    
dt = GradientBoostingClassifier(**gbr_params)
dt.fit(X_train, y_train)
predictions.append(dt.predict(X_test))

knn = XGBClassifier(**xgb_params)
knn.fit(X_train, y_train)
predictions.append(knn.predict(X_test))

gbr = CatBoostClassifier(**cats_params)
gbr.fit(X_train, y_train)
predictions.append(gbr.predict(X_test))

lgbm = LGBMClassifier(**lgbm_params)
lgbm.fit(X_train, y_train)
predictions.append(lgbm.predict(X_test))

for i, preds in enumerate(predictions):
    x_val_with_metapreds[:, -(i+1)] = preds

0:	total: 43.7ms	remaining: 1m 27s
300:	total: 8.41s	remaining: 47.5s
600:	total: 15.8s	remaining: 36.7s
900:	total: 23s	remaining: 28.1s
1200:	total: 30.2s	remaining: 20.1s
1500:	total: 37.5s	remaining: 12.5s
1800:	total: 44.7s	remaining: 4.93s
1999:	total: 49.5s	remaining: 0us


In [49]:
lr = LogisticRegression()
lr.fit(x_train_with_metapreds, y_train)
lr_preds_train = lr.predict(x_train_with_metapreds)
lr_preds_val = lr.predict(x_val_with_metapreds)

train_auc_values['lr'] = roc_auc_score(y_train, lr_preds_train)
test_auc_values['lr'] = roc_auc_score(y_test, lr_preds_val)

In [51]:
auc_scores = pd.concat([pd.Series(train_auc_values, name='train'), 
                        pd.Series(test_auc_values, name='val')], 
                       axis=1)
auc_scores

,train,val
grb,0.507082,0.501565
xgb,0.509707,0.502112
cats,0.504395,0.500000
lgbm,0.505946,0.500000
lr,0.500000,0.500000


In [36]:
a = pd.read_csv('grb_sub3.csv')
b = pd.read_csv('stacker.csv')
c = pd.read_csv('xg_subfull.csv')
d = pd.read_csv('xg2.csv')

In [37]:
# Add all to test
new_test = test_df.copy()

In [57]:
grb = a.loc[:,'target']
stack = b.loc[:, 'target']
xg = c.loc[:, 'target']
xg2 = d.loc[:, 'target']

In [62]:
test_df['grb'] = grb
new_test['stack'] = stack
new_test['xg'] = xg
new_test['xg2'] = xg2

In [63]:
test_df.head(2)

,city,city_development_index,experience,last_new_job,training_hours,experience_lag1,experience_lag2,experience_lag3,training_hours_lag1,training_hours_lag2,...,company_size_local,company_size_multinational,company_size_start-up,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,grb
18359,73,0.448,0.998004,-9999,0.041791,20.0,20.0,11.0,84.0,41.0,...,0,0,0,0,0,0,0,1,0,NaN
18360,70,0.939,1.000000,1,0.277612,0.0,20.0,20.0,15.0,84.0,...,0,0,1,0,0,0,0,0,1,NaN


In [56]:
a.loc[:,'target']

0        0.937498
1        0.075016
2        0.320841
3        0.069689
4        0.091511
           ...   
15016    0.101695
15017    0.242853
15018    0.082271
15019    0.245319
15020    0.244478
Name: target, Length: 15021, dtype: float64

In [70]:
grb.fit(X,Y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2020, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [71]:
xgb.fit(X, Y, verbose=False)

XGBClassifier(base_score=0.7, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.03, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbose=300, verbosity=1)

In [39]:
grb.fit(X, Y)

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.02, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=700, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [73]:
test_df = test_df.drop('grb', axis=1)

In [77]:
# make a test prediction
prediction = grb.predict_proba(test_df)[:,1]

In [78]:
sample['target'] = prediction

In [76]:
sample.to_csv('xgblate.csv', index=False)

In [79]:
sample.to_csv('grb_late.csv', index=False)

In [71]:
a = pd.read_csv('grb_late.csv')
b = pd.read_csv('xgblate.csv')

In [80]:
stacked = (a['target'] * 4 + b['target']*.6) / 2

In [81]:
sample['target'] = stacked
sample.to_csv('stacker_late.csv', index=False)